In [6]:
from IPython.display import HTML, SVG, display
from splinter import Browser
import random

In [2]:
class Node:
    def __init__(self, index, x, y, z, adjacent=None):
        self.index = index  # unique node identifier
        self.x = x
        self.y = y
        self.z = z
        self.adjacent = set() if adjacent is None else adjacent

    def __hash__(self):
        return self.index

    def __eq__(self, other):
        """
        This will make any two nodes with the same coordinates or the same
        index equal to one another.

        Having this function defined in conjuction with self.__hash__()
        makes it so we can use this class as a key in a Python dictionary
        """
        return self.index == other.index or (self.x, self.y, self.z) == (
            other.x, other.y, other.z)

    def __ne__(self, other):
        return not (self == other)

    def __repr__(self):
        """
        Just a convenience function for us to be able to represent the Node in
        our shell.
        """
        return "{}: {}".format(self.index,
                               ','.join(map(str, (self.x, self.y, self.z))))

    def connect_to(self, node):
        self.adjacent.add(node)


class Graph:
    def __init__(self, nodes=None):
        self.nodes = set() if nodes is None else nodes

    def __repr__(self):
        """
        Returns a string displaying the graph as an adjacency list.
        """
        lines = []

        for node in sorted(self.nodes, key=lambda n: n.index):
            lines.append('{}: {}'.format(node.index, ','.join(
                map(str, (n.index for n in node.adjacent)))))

        return '\n'.join(lines)

    def __str__(self):
        """
        Represents graph in string format needed by Network visualizer.
        """

        # creates a list of comma-separated x, y, z pairs for each node
        coordinates = [
            ', '.join(map(str, (n.x, n.y, n.z))) for n in self.nodes
        ]

        # a collection of comma-separated (from: Node, to: Node) pairs.
        # since the Graph is represented as an adjacency list
        # with each node having a collection of neighbors as an
        # attribute, the comprehension is nested.
        edges = [
            ', '.join(map(str, (f.index, t.index)))
            for f in self.nodes for t in f.adjacent
        ]

        return '\n'.join(coordinates) + '\n\n' + '\n'.join(edges)

    def add_node(self, node):
        if node in self.nodes:
            print('node with index {} already exists'.format(node.index))
        else:
            self.nodes.add(node)

    def connect(self, from_, to):
        if from_ in self.nodes and to in self.nodes:
            for node in self.nodes:
                if node == from_:
                    node.adjacent.add(to)

    def seed(self, n=10):
        """n: the number of nodes we want to seed the graph with"""

        # populate nodes
        self.nodes = {
            # insert nodes with random x, y, z coordinates in the
            # range(0, 5) - not inclusive
            Node(i, *(random.randrange(5) for _ in range(3)))
            for i in range(n)
        }

        # add edges to nodes
        for node in self.nodes:
            k = random.randrange(len(self.nodes))
            adjacent = {n for n in random.sample(self.nodes, k) if n != node}
            node.adjacent = adjacent


graph = Graph()

graph.seed()

data = str(graph)

In [7]:
browser = Browser(driver_name='phantomjs')
browser.visit('https://geo.improbable.io/solutions-networkviewer/')
browser.find_by_id('dataArea').fill(data)
browser.find_by_id('process').click()
svg = browser.find_by_id('view').html.strip()

display(SVG(svg))